In [ ]:
#### Installs ####
# perform in terminal:
# pip install selenium

# ChromeDriver download
# https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.264/win64/chromedriver-win64.zip

# Imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import sqlite3
from pathlib import Path
import string
import my_utils
import random
import os

In [ ]:
# Initialize driver 
# Specify the path to your chromedriver
# NOTE: likely to throw errors if chromedriver and chrome versions differ
# current chrome version: Version 135.0.7049.42 (Official Build) (64-bit)
# to download most recent chrome driver, go to: https://googlechromelabs.github.io/chrome-for-testing/
# https://storage.googleapis.com/chrome-for-testing-public/135.0.7049.42/win64/chromedriver-win64.zip
#driver_path = "C://Users//jsk33//OneDrive//Github//scrape-addgene//chromedriver-win64//chromedriver.exe"  # Replace with the actual path to the WebDriver
driver_path = ".//chromedriver-win64//chromedriver-win64//chromedriver.exe"
service = Service(driver_path)

# Initialize WebDriver
#driver = webdriver.Chrome(service=service)

In [ ]:
# create database if doesnt exist
# instantiate

# path to database file
db_path = Path("addgene_database.db")

# check if file exists, if not, create it
if not db_path.is_file():
    print(f"File '{db_path}' does not exist. Creating database file...")
    conn = sqlite3.connect('addgene_database.db')
else:
    print(f"File '{db_path}' already exists.")
    conn = sqlite3.connect('addgene_database.db')

#
#cursor = conn.cursor()

# Create table
#cursor.execute('''CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')

# Insert a row
#cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('Alice', 25))

# Query data
#cursor.execute('SELECT * FROM users')
#print(cursor.fetchall())

#conn.commit()
#conn.close()


In [ ]:
#### Set search parameters
target_plasmid_IDs = []

## read the list of terms in
search_par_file = "./combined_search_parameters.txt"
search_par = set()

# automatically deduplicated
with open(search_par_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        search_par.update(lines)

# convert to list and randomize to improve search efficiency
shuffled_par = random.sample(list(search_par), k=len(search_par))

## write randomized parameters to file
shuffled_path = "./shuffled_parameters.txt"

if not os.path.exists(shuffled_path):
    with open(shuffled_path, "w") as f:
        for item in shuffled_par:
            f.write(item + "\n")
    print(f"✅ File created: {shuffled_path}")
else:
    print(f"⚠️ File already exists: {shuffled_path}")

In [ ]:
#=== Scan again using the combined search parameters including model organism gene names and snapgene common features ===#

## read the list of terms in
shuffled_path = "./shuffled_parameters.txt"
shuffled_par = []
with open(shuffled_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        shuffled_par.extend(lines)

####
my_utils.scan_addgene(shuffled_par)


In [ ]:
#### Once search for IDs is complete, deduplicate, sort, and output final file
## Save results
raw_file = "./addgene_IDs.txt"
raw_IDs = set()

# 
with open(raw_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        raw_IDs.update(lines)

# convert to list and randomize to improve search efficiency
dedup_sorted_IDs = sorted(list(raw_IDs))

## write randomized parameters to file
dedup_IDs_file = "./final_addgene_IDs.txt"

if not os.path.exists(dedup_IDs_file):
    with open(dedup_IDs_file, "w") as f:
        for item in dedup_sorted_IDs:
            f.write(item + "\n")

In [ ]:
#=== Read in IDs and initiate scraping of plasmid sequences ===#

# read in the plasmid IDs
final_IDs_path = "./final_addgene_IDs.txt" 
plasmid_IDs = [] 

#
with open(final_IDs_path, 'r') as f:
    lines = [line.strip() for line in f if line.strip()]
    plasmid_IDs.extend(lines)

####
my_utils.scrape_plasmid_data(plasmid_IDs, reset=True)